In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import seaborn as sns
import matplotlib.pyplot as plt
import glob
import os
import xgboost as xgb
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from tqdm.notebook import tqdm
from sklearn.feature_selection import SelectPercentile
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neighbors import BallTree
from sklearn.svm import SVR
from sklearn.linear_model import Lasso,Ridge
from sklearn.ensemble import RandomForestRegressor
from joblib import dump, load
from joblib import Parallel, delayed
from sklearn.model_selection import cross_val_score

In [2]:


def train_test_split(df, metropole=None,type_local=None, random_state=42,split=True, trimestres=['2021-T3','2021-T4','2022-T1','2022-T2'],quartile=None):
    if quartile:
      df=df[df.prix_m2_actualise<df.prix_m2_actualise.quantile(1-quartile)][df.prix_m2_actualise>df.prix_m2_actualise.quantile(quartile)]
    if metropole:
          df = df[df['LIBEPCI'].str.contains(metropole)]
    if type_local:
          df = df[df['type_local'].str.contains(type_local)]



    # shuffle entire dataframe
    shuffled_df = df.sample(frac=1, random_state=random_state)
    train_df = shuffled_df[~shuffled_df['trimestre_vente'].isin(trimestres)]
    test_df = shuffled_df.drop(train_df.index)
    if split:
      # select training data
     
      train_x = train_df.drop('prix_m2_actualise', axis=1)
      train_y = train_df['prix_m2_actualise']

      #select test data
      test_x = test_df.drop('prix_m2_actualise', axis=1)
      test_y = test_df['prix_m2_actualise'] ### pour rester coherent on utise les prix  non actualisé pour la prediction
      return train_x, test_x, train_y, test_y
    else:
      test_df['prix_m2_actualise']=test_df['prix_m2']
      return pd.concat([train_df,test_df],axis=0)

In [3]:

def preprocessing(data,type_local):
    to_drop=['adresse_numero', 'adresse_suffixe','numero_disposition',
       'adresse_code_voie', 'code_postal', 'code_commune',
       'ancien_code_commune','ancien_nom_commune' , 'ancien_id_parcelle',
       'numero_volume', 'lot1_numero', 'lot1_surface_carrez', 'lot2_numero',
       'lot2_surface_carrez', 'lot3_numero', 'lot3_surface_carrez',
       'lot4_numero', 'lot4_surface_carrez', 'lot5_numero',
       'lot5_surface_carrez','code_type_local',
       'code_nature_culture', 'nature_culture', 'code_nature_culture_speciale',
       'nature_culture_speciale','id_mutation','id_parcelle','numero_disposition', 'nature_mutation','valeur_fonciere',
       'id_parcelle','nature_mutation','date_mutation','LIBEPCI','prix_actualise','DCOMIRIS','DCIRIS','prix_m2',
       'type_local','geometry','indices','quantile_prix','coeff_actu']
    drop_clean=list(set(data.columns)&set(to_drop))
    data.drop(drop_clean,axis=1,inplace=True)
    numerical_columns = list(data.select_dtypes(exclude=["object","string"]).columns)
    target='prix_m2_actualise'
    numerical_columns=[col for col in numerical_columns if col!=target]
    corr_matrix = data[numerical_columns].corr().abs()
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
    to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]
    print(to_drop)
    data.drop(to_drop,axis=1,inplace=True)
    if type_local=="Appartement":
        data.drop(['surface_terrain'],axis=1,inplace=True)
    #data[['prix_m2_actualise','prix_m2_zone']]=np.log(data[['prix_m2_actualise','prix_m2_zone']])
    data.dropna(inplace=True)
    return data

                              
    

In [ ]:
def build_pipeline(model,data):
        numerical_columns = list(data.select_dtypes(exclude=["object","string"]).columns)
        target='prix_m2_actualise'
        numerical_columns=[col for col in numerical_columns if col!=target]
        categorical_columns=['code_departement'] 
        folds = 3
        numeric_transformer = Pipeline(
            steps=[("imputer", SimpleImputer(strategy="median")), ("scaler", StandardScaler())]
                )

        categorical_transformer = Pipeline(
            steps=[
                ("encoder", OneHotEncoder(handle_unknown="ignore"))
            ]
            )
        preprocessor = ColumnTransformer(
            transformers=[
                ("num", numeric_transformer,numerical_columns),
                ("cat", categorical_transformer,categorical_columns),
            ])
        rd=RandomForestRegressor(n_jobs=-1)
        xg_reg = xgb.XGBRegressor(n_jobs=-1)
        if model=='linear':
            params = {"poly_features__degree": [1,2],'model__alpha': np.arange(0, 0.2, 0.01)}
            poly_pipeline = Pipeline([('preprocessor',preprocessor),('poly_features', PolynomialFeatures()), ('model', Lasso())])
            clf = GridSearchCV(poly_pipeline, cv=folds, scoring='r2', param_grid=params,n_jobs=-1)
        elif model=='xgboost':
            params = {'xg__eta': [0.3, 0.02],'xg__n_estimators': [100,500,1000,5000]}
            xg = xgb.XGBRegressor(n_jobs = -1)
            xg_reg = Pipeline(
            steps=[("preprocessor", preprocessor), ("xg", xg)],verbose=True)
            clf = RandomizedSearchCV(xg_reg, param_distributions=params, n_iter=2,
            scoring='r2', n_jobs=-1, cv=folds,random_state=1001 )
        else:
                rd_pipeline = Pipeline(
                    steps=[("preprocessor", preprocessor), ("rd", rd)]
                )
                params = {
                 'rd__n_estimators':[100,150,200,300]
                }
                clf= RandomizedSearchCV(rd_pipeline, param_distributions=params, n_iter=2,
                        scoring='r2', n_jobs=-1, cv=folds,
                                       random_state=1001 )

        
            
        return clf              


In [ ]:
def train_score_save(model,data,metropole,type_local):
        data=train_test_split(data,metropole,type_local,split=False)
        data=preprocessing(data,type_local)
        shape=data.shape
        clf=build_pipeline(model,data)
        X_train, X_test, y_train, y_test =train_test_split(data,split=True,quartile=0.05)
        clf.fit(X_train,y_train)
        best_score=clf.best_score_
        best_params=clf.best_params_
        clf.fit(X_train,y_train)
        y_predict=clf.predict(X_test)
        rmse=mean_squared_error(y_predict,y_test,squared=False)
        score=r2_score(y_predict,y_test)
        error=pd.DataFrame((np.abs(y_test-y_predict)/y_test)*100)
        median=error.median()[0]
        mean=error.mean()[0]
        shape=data.shape
        estimator = clf.best_estimator_
        dump(estimator, "results/{}-{}-{}.joblib".format(metropole.strip().replace(" ",'-'),type_local,model))
        result="{}-{}-{}-best_score: {},best_param :{} rmse:{} ,score:{},median: {}, mean:{}".format(metropole,type_local,model,best_score,best_params,rmse,score,str(median),str(mean))
        dataframe=pd.DataFrame([[metropole,type_local,model,best_score,best_params,rmse,score,median,mean,shape]],columns=['metropole','type_local','model','best_score_cv_search','best_params','rmse','score_r2_test','error_prix_actualise_median','error_prix_actualise_mean','shape'])
        f = open("results_log/results.txt","a+")
        f.write(result+'\n')
        f.close()
        return dataframe

    
    
   
    

In [4]:
data_final=pd.read_csv('Final.csv')

FileNotFoundError: ignored

In [ ]:
# train_score_save(data=data_final,model='svr',metropole='Métropole du Grand Paris',type_local="Appartement")

C:\Users\ntedonze\AppData\Local\Temp\2\ipykernel_60576\258786554.py:19: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))


,metropole,type_local,shape
0,Métropole du Grand Paris,Appartement,"(357999, 38)"


In [ ]:
def fonc(params):
    (metro,type_local,mod)=params
    return train_score_save(data=dataa,model=mod,metropole=metro,type_local=type_local)

In [ ]:
#### Main to run model


In [ ]:
metropole=list(data_final.LIBEPCI.unique())
type_bien=["Appartement","Maison"]
modele=['linear','xgboost','random_forest']
permutation=[(i,j,k) for i in metropole for j in type_bien for k in modele ]
dataa=data_final.copy(deep=False)


In [ ]:
results = Parallel(n_jobs=-1, verbose=1)\
    (delayed(fonc)(params) for params in permutation)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 40 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of  20 | elapsed:   38.8s remaining:  2.6min
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:  1.0min finished


In [ ]:
temp=pd.concat(results,axis=0)
temp.to_csv('results_log/results_dataframe.csv',index=False)### save result on disk

In [ ]:

with open("results/results.txt") as f:
    lines = f.readlines()

In [ ]:
!pip install parse
import parse

In [ ]:
# from parse import compile
# result="{}-{}-{}-best_score: {},best_param :{} rmse:{} ,score:{},median: {}, mean:{}"
# format_string = result
# p = compile(result)
# results_dataframe=[]
# for k in range(int(len(lines)/3)):
#     string=''.join(lines[3*k:3*k+3]).replace("\n", '')
#     string=string.replace('-Marseille-Provence','Marseille Provence')
#     result_list=list(p.parse(string))
#     result_list[-2]=result_list[-2].split()[1].replace('dtype:','')
#     result_list[-1]=result_list[-1].split()[1].replace('dtype:','')
#     result_list[0]=result_list[0].replace('Marseille Provence','-Marseille-Provence')
#     results_dataframe.append(pd.DataFrame([result_list],columns=['metropole','type_local','model','best_score_cv_search','best_params','rmse','score_r2_test','error_prix_actualise_median','error_prix_actualise_mean']))
    

In [ ]:
# result=pd.concat(results_dataframe)


In [ ]:
# result_final=pd.merge(result,temp ,on=['metropole','type_local'])
# result_final["error_prix_actualise_median"]=result_final["error_prix_actualise_median"].astype('float')
# result_final=result_final.sort_values(by='error_prix_actualise_median',ascending=True)
# result_final

,metropole,type_local,model,best_score_cv_search,best_params,rmse,score_r2_test,error_prix_actualise_median,error_prix_actualise_mean,shape
58,Métropole du Grand Paris,Appartement,random_forest,0.7889571055599807,{'rd__n_estimators': 200},1469.3859743965388,0.7423629152665966,9.208510e+00,14.055572,"(357999, 38)"
19,Bordeaux Métropole,Appartement,random_forest,0.6408777404973346,{'rd__n_estimators': 200},695.8283635039036,0.44514535098081587,9.417081e+00,13.047243,"(35602, 38)"
20,Bordeaux Métropole,Appartement,xgboost,0.6495437431689904,"{'xg__n_estimators': 1000, 'xg__eta': 0.02}",688.3358215017068,0.4034513092106353,9.595096e+00,13.117453,"(35602, 38)"
59,Métropole du Grand Paris,Appartement,xgboost,0.8017154129482176,"{'xg__n_estimators': 1000, 'xg__eta': 0.02}",1464.7092338889518,0.7316805147523238,9.740180e+00,14.451721,"(357999, 38)"
32,Montpellier Méditerranée Métropole,Appartement,xgboost,0.5963711342434251,"{'xg__n_estimators': 1000, 'xg__eta': 0.02}",540.1363314129751,0.2699688276360993,9.885208e+00,12.87503,"(26170, 38)"
16,Toulouse Métropole,Appartement,random_forest,0.6393294179334118,{'rd__n_estimators': 200},556.4271536321863,0.4130179601384618,9.966724e+00,13.139864,"(42840, 40)"
31,Montpellier Méditerranée Métropole,Appartement,random_forest,0.5790496147892428,{'rd__n_estimators': 200},545.4926699707559,0.3425310781810861,1.005132e+01,12.900276,"(26170, 38)"
17,Toulouse Métropole,Appartement,xgboost,0.6497477657784455,"{'xg__n_estimators': 1000, 'xg__eta': 0.02}",560.027774047436,0.3277408269541918,1.023776e+01,13.312381,"(42840, 40)"
34,Métropole d'Aix-Marseille-Provence,Appartement,random_forest,0.6153162579271652,{'rd__n_estimators': 200},641.3748129506541,0.5264597979096547,1.051291e+01,14.953197,"(76357, 37)"
46,Métropole Européenne de Lille,Appartement,random_forest,0.7174573957531596,{'rd__n_estimators': 200},555.9868691571128,0.5600195577780773,1.059651e+01,13.825753,"(30865, 36)"


In [ ]:
# result_final.to_csv("machine_learning.csv",index=False)

In [ ]:
# clf=load("results/Bordeaux-Métropole-Appartement-random_forest.joblib")

In [ ]:
def feature_importance_graph(model,data,metropole,type_local):
      '''
      Generate and save feature importance
      '''
        data=train_test_split(data,metropole,type_local,split=False)
        data=preprocessing(data,type_local)
        numerical_columns = list(data.select_dtypes(exclude=["object","string"]).columns)
        target='prix_m2_actualise'
        numerical_columns=[col for col in numerical_columns if col!=target]
        clf=load("results/{}-{}-{}.joblib".format(metropole.strip().replace(" ",'-'),type_local,model))
        new_cat_cols = clf.named_steps['preprocessor'].named_transformers_["cat"].named_steps["encoder"].get_feature_names_out(['code_departement','trimestre_vente'])
        model_ = clf[-1]
        # making a pandas dataframe
        importance=list(zip(numerical_columns+list(new_cat_cols), model_.feature_importances_))
        df_importances = pd.DataFrame(importance,columns=['Feature', 'Importance']).sort_values(by='Importance', ascending=True)
        df_importances=df_importances[~df_importances.Feature.str.startswith('trimestre')]
        df_importances.Importance = (df_importances.Importance / sum(df_importances.Importance)) * 100
        plt.figure(figsize=(8,20))
        plt.barh(data=df_importances,y='Feature', width='Importance',color='#ff9600')
        y=list(df_importances.Importance)
        for i in range(len(y)):
            plt.text(x= round(y[i],2),y= i,s= round(y[i],2), c='b')
        plt.xlabel('feature_importance(%)')
        plt.ylabel('features')
        plt.title('Analysis of feature importance for model:{}-{}-{}'.format(model,metropole,type_local))
        plt.savefig('images/{}-{}-{}.png'.format(model,metropole,type_local), bbox_inches='tight')
     

In [ ]:
metropole=list(data_final.LIBEPCI.unique())
type_bien=["Appartement","Maison"]
modele=['xgboost','random_forest']
permutation=[(i,j,k) for i in metropole for j in type_bien for k in modele ]
dataa=data_final.copy(deep=False)

In [ ]:
def fonc(params):
    (metro,type_local,mod)=params
    return feature_importance_graph(data=dataa,model=mod,metropole=metro,type_local=type_local)

In [ ]:
results = Parallel(n_jobs=-1, verbose=1)\
    (delayed(fonc)(params) for params in permutation)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 40 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of  40 | elapsed:   25.2s remaining:  8.0min
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:  2.3min finished


In [ ]:
# clf[0]

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', StandardScaler())]),
                                 ['code_departement', 'nombre_lots',
                                  'surface_reelle_bati',
                                  'nombre_pieces_principales', 'latitude',
                                  'prix_m2_zone', 'moyenne', 'moyenne_brevet',
                                  'Banques', 'Bureaux_de_Poste', 'Commerces',
                                  'Ecoles', 'Collèges_Lycées', 'Medecins',
                                  'Gar...
                                  'Espaces_remarquables_et_patrimoine',
                                  'Taux_pauvreté_seuil_60', 'Q1', 'Mediane',
                                  'Ecart_inter_Q_rapporte_a_la_mediane', 'D1',
                                  'D9', 'Rapport_interdécile_D9/D1', 'Gini',
                                  'Part_revenus_activite', 'Part_salaire',
                                  'Part_revenus_chomage',
                                  'Part_revenus_non_salariées', ...]),
                                ('cat',
                                 Pipeline(steps=[('encoder',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['code_departement', 'trimestre_vente'])])

In [ ]:
def generate_error_test_graph(model,data,metropole,type_local,save_error_plot=True):
        data=train_test_split(data,metropole,type_local,split=False)
        data=preprocessing(data,type_local)
        X_train, X_test, y_train, y_test =train_test_split(data,split=True,quartile=0.05)
        numerical_columns = list(data.select_dtypes(exclude=["object","string"]).columns)
        target='prix_m2_actualise'
        numerical_columns=[col for col in numerical_columns if col!=target]
        clf=load("results/{}-{}-{}.joblib".format(metropole.strip().replace(" ",'-'),type_local,model))
        y_predict=clf.predict(X_test)
        error=np.abs(y_test-y_predict)*100/y_test
        plt.figure(figsize=(10,5))
        plt.hist(error,color='#ff9600',bins=100)
        plt.xlabel("taux d'erreur dans la prediction de nos prix")
        plt.xlabel("Taux erreur")
        plt.ylabel("nombre de biens")
        plt.title('Error for  model:{}-{}-{}'.format(model,metropole,type_local))
        plt.savefig('images/error-{}-{}-{}.png'.format(model,metropole,type_local), bbox_inches='tight')    
        # result=pd.concat([X_test.trimestre_vente.reset_index(),y_test.reset_index(),pd.Series(y_predict,name="y_predict").reset_index()],axis=1)
        # result=result[['trimestre_vente',"prix_m2_actualise",'y_predict']]
        # result['metropole']=metropole
        # result['type_local']=type_local
        return None
        

     

In [ ]:
metropole=list(data_final.LIBEPCI.unique())
type_bien=["Appartement","Maison"]
modele=['xgboost','linear','random_forest']
permutation=[(i,j,k) for i in metropole for j in type_bien for k in modele ]
dataa=data_final.copy(deep=False)

In [ ]:
def fonc(params):
    (metro,type_local,mod)=params
    return generate_test_data(data=dataa,model=mod,metropole=metro,type_local=type_local)

In [ ]:
results_dataframe = Parallel(n_jobs=-1, verbose=1)\
    (delayed(fonc)(params) for params in permutation)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 40 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 out of  60 | elapsed:  2.1min remaining:   54.8s
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:  3.0min finished


In [ ]:
# result=pd.concat(results_dataframe)

In [ ]:
# result=result.rename(columns={'prix_m2_actualise':'prix_m2_test'})


In [ ]:
# result.to_csv("test_data_predict.csv",index=False)

C:\Users\ntedonze\AppData\Local\Temp\2\ipykernel_60576\1635650492.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df=df[df.prix_m2_actualise<df.prix_m2_actualise.quantile(1-quartile)][df.prix_m2_actualise>df.prix_m2_actualise.quantile(quartile)]
